# Neuronale Netze mit Lösungen

In diesem Jupyter Notebook soll das Back-Propagation Netzwerk aus der Vorleung von Grund auf implimentiert werden. Zwischen der zeidimensionalen Eingabe- und der eindimensionalen Ausgabeschicht befinden sich zwei weitere Schichten. Die erste dieser Schichten hat eine Breite von vier, während die zweite Schicht eine Breite von zwei aufweist.

## Laden nötiger Module

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Definieren nötiger Funktionen und aufbau des Netzwerks

Im Rahmen des Trainings des Neuronalen Netzes treten einige Funktion häufig auf, so dass es sinnvoll ist, diese im Vorfeld zu definieren. Dazu zählen die Sigmoid-Funktion mit Vertärker (`amplification`), deren Ableitung, die Theta-Funktion, welche für das schlussendliche Modell verwendet wird und die Ableitung der Kreuzentropie nach der Hypothese.

__Aufgabe__: Implimentiere diese Funktionen im unten stehenden Code-Gerüst, so dass sie auch Vektoren entgegen nehmen können.

In [ ]:
def sig(x):    # Sigmoid-Funktion mit Verstärker
    global amplification
    return 1/(1+np.exp(-amplification*x))


def sig_prime(x):    # Ableitung der Sigmoid-Funktion
    global amplification
    return sig(x)*(1-sig(x))*amplification


def theta(x):    # Theta-Funktion (1, wenn Argument größer oder gleich Null; 0 sonst)
    return x >= 0


def dR_dh(h, y):    # Ableitung der Kreuzentropie nach der Hypothese h
    return np.nan_to_num((h-y)/(h*(1-h)), posinf = 0, neginf = 0)

In jeder Schicht $l$ wird ein Vektor $\vec{z}^{(l)}$ aus der Ausgabe der vorangegangenen Schicht $\vec{x}^{(l-1)}$ durch die Abbildung
$$\vec{z}^{(l)}=\underline{W}^{(l)}\vec{x}^{(l-1)}+\vec{w}^{(l)}$$
mit einer Matrix $\underline{W}^{(l)}$ und einem Verschiebungsvektor $\vec{w}^{(l)}$ konstruiert. Zur Ausgabe der Schicht $l$ wird dann die Aktivierungsfunktion $\phi^{(l)}$ auf $\vec{z}^{(l)}$ angewandt. Hier soll $\phi^{(l)}$ komponentenweise die Sigmoidfunktion anwenden.

__Aufgabe__: Vervollständige die unten stehenden Funktionen zur Bestimmung der einzelnen $\vec{z}^{(l)}$. Beachte dabei auch die oben beschriebene Struktur der einzelnen Schichten.

In [ ]:
def z1(x0):    # Aktivierungsvektor der ersten Schicht
    global w1M, w1V    # 4x2-Matrix und Verschiebungsvektor
    return w1M@x0+w1V


def z2(x1):    # Aktivierungsvektor der zweiten Schicht
    global w2M, w2V    # 2x4-Matrix und Verschiebungsvektor
    return w2M@x1+w2V


def z3(x2):    # Aktivierungsvektor der dritten Schicht
    global w3M, w3V    # 1x2-Matrix als Vektor und Verschiebungsvektor in Form einer Zahl
    return w3M.T@x2+w3V    

## Trainieren des Neuronalen Netzes

Im Datensatz `neural_network_data01.dat` findest Du einen linear separierbaren Datensatz mit $d$ Datenpunkten.

__Aufgabe__: Lade den Datensatz und trage ihn auf. Formatiere den Datensatz dann so, dass die Koordinaten in Paaren $(x_1, x_2)$ in einem Vektor `X0` der Länge $d$ vorliegen.

In [ ]:
# Laden des Datensatzes
x1, x2, y = np.loadtxt('neural_network_data01.dat', unpack = True)

# Auftragen des Datensatzes
plt.scatter(x1, x2, c = y, cmap = 'seismic')

plt.grid(True, color = 'grey', linestyle = '--')
plt.axis('equal')

plt.show()

In [ ]:
# Formatieren des Datensatzes
d = len(y)    # Bestimmen der Anzahl der Datenpunkte
X0 = np.zeros(shape = (d, 2))    # Erstellen eines leeren Arrays der passenden Form
for n in range(d):
    X0[n] = np.array([x1[n], x2[n]])    # Auffüllen des Arrays mit den entsprechenden Daten

## Initialisieren der Gewichte
Um das Netzwerk zu trainieren, müssen zunächst die Gewichte initialisiert werden. Für die lineare Regression reicht es meist aus, alle Gewichte mit Null zu initialiseren. Für komplexere Aufgaben kann es hilfreich sein, die Gewichte radnomisiert zu initialisieren.

__Aufgabe__: Vervollständige das nachstehende Code-Gerüst, um die Gewichte zu initialisieren. Achte dabei vor allem auf die oben beschriebene Struktur des neuronalen Netzes

In [ ]:
# Schicht 1
w1V = np.zeros(4)    # Verschiebungsvektor der Dimension 4
w1M = np.zeros(shape = (4, 2))   # 4x2-Matrix

# Schicht 2
w2V = np.zeros(2)   # Verschiebungsvektor der Dimension 2
w2M = np.zeros(shape = (2, 4))    # 2x4-Matrix

# Schicht 3
w3V = 0    # Verschiebungsvektor der Dimension 1 -> Zahl
w3M = np.zeros(2)    # 1x2-Matrix, hier dargestellt als Vektor

# Trainieren des Neuronalen Netzes

Um ein Backpropagation-Netzwerk zu trainiere, werden in jedem Durchlauf mit den aktuellen Gewichten, die Daten durch das Netz gereicht und dabei alle Vektoren $\vec{z}^{(l)}$ und $\vec{x}^{(l)}$ gespeichert. Dies ist der _Forward Pass_. Anschließend werden von der letzten Schicht beginnend die Gradienten bzgl. der einzelnen Gewichte errechnet. Da in den Gradienten der ersten Schichten auch Ausdrück der Gradienten der späteren Schichten auftauchen, können die Gradienten der frühen Schichten so wesentlich effizient berechnet werden. Dies ist der _Backward Pass_. Schlussendlich werden mit dem Gradientenabstiegsverfahren die neuen Gewichte bestimmt. Dieser Prozess wird für mehrere Epochen wiederhohlt.

In der Vorlesung haben wir gezeigt, dass die Gradienten für einen einzelnen Datenpunkt für ein Netzwerk der obigen Struktur die Form
$$
\frac{\partial R}{\partial w^{(3)}}=\frac{\partial R}{\partial h}\mathrm{sig}'(z^{(3)})\quad\quad \frac{\partial R}{\partial\underline{W}^{(3)}_i}=\frac{\partial R}{\partial w^{(3)}}x^{(2)}_i\\
\frac{\partial R}{\partial w^{(2)}_i} = \frac{\partial R}{\partial w^{(3)}}\underline{W}_i^{(3)}\mathrm{sig}'(z_i^{(2)})\quad\quad \frac{\partial R}{\partial \underline{W}^{(2)}_{ij}}=\frac{\partial R}{\partial w^{(2)}_i}x_j^{(1)}\\
\frac{\partial R}{\partial w_{i}^{(1)}} = \left[\left(\nabla_{\vec{w}^{(2)}}R\right)^T\underline{W}^{(2)}\right]_i\mathrm{sig}'(z_i^{(1)})\quad\quad \frac{\partial R}{\partial \underline{W}_{ij}^{(1)}}=\frac{\partial R}{\partial w_i^{(1)}}x_j^{(0)}
$$
haben. Die tatsächlichen Gradienten ergeben sich wegen
$$R = \frac{1}{N}\sum_{k = 1}^{d}\mathrm{Kreuzentropie}(y_k, \vec{x}_k^{(0)})$$
dann als Summe dieser Terme.

__Aufgabe__: Vervollständige das nachstehende Code-Gerüst eines Neuronalen Netzes. 

In [ ]:
# Einstellen der Hyperparameter
amplification = 10
epochen = 1000
eta = 10**(-3)

In [ ]:
# Trainieren des Neuronalen Netz
for i in range(epochen):    # Iterieren über die Epochen
    
    ### Forward Pass ###
    
    # Schicht 1
    Z1 = np.zeros(shape = (d, 4))    # Erstellen eines leeren Arrays zum Speichern von Z1 für jeden Datenpunkt
    for n in range(d):
        Z1[n] = z1(X0[n])    # Ermitteln von Z1 für jeden Datenpunkt
    X1 = sig(Z1)    # Ermitteln von X1 für jeden Datenpunkt
    
    
    # Schicht 2
    Z2 = np.zeros(shape = (d, 2))    # Erstellen eines leeren Arrays zum Speichern von Z2 für jeden Datenpunkt
    for n in range(d):
        Z2[n] = z2(X1[n])    # Ermitteln von Z2 für jeden Datenpunkt
    X2 = sig(Z2)    # Ermitteln von X2 für jeden Datenpunkt
    
    
    # Schicht 3
    Z3 = np.zeros(d)    # Erstellen eines leeren Arrays zum Speichern von Z3 für jeden Datenpunkt
    for n in range(d):
        Z3[n] = z3(X2[n])    # Ermitteln von Z3 für jeden Datenpunkt
    X3 = sig(Z3)    # Ermitteln von X3 für jeden Datenpunkt
    
    
    
    
    
    ### Backward Pass ###
    
    # Schicht 3
    # Gradient des Verschiebungsvektors
    dR_dw3V_temp = dR_dh(X3, y)*sig_prime(Z3)    # Bestimme einen Array mit den Gradienten der einzelnen Datenpunkten
    dR_dw3V = np.sum(dR_dw3V_temp)    # Summiere die einzelnen Gradienten auf
    
    # Gradient der Matrix
    dR_dw3M_temp = np.zeros(shape = (d, 2))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw3M_temp[n] = dR_dw3V_temp[n]*X2[n]    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw3M = np.sum(dR_dw3M_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    
    # Schicht 2
    # Gradient des Verschiebungsvektors
    dR_dw2V_temp = np.zeros(shape = (d, 2))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw2V_temp[n] = dR_dw3V_temp[n]*w3M*sig_prime(Z2[n])    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw2V = np.sum(dR_dw2V_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    # Gradient der Matrix
    dR_dw2M_temp = np.zeros(shape = (d, 2, 4))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw2M_temp[n] = np.outer(dR_dw2V_temp[n], X1[n])    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw2M = np.sum(dR_dw2M_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    
    # Schicht 1
    # Gradient des Verschiebungsvektors
    dR_dw1V_temp = np.zeros(shape = (d, 4))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw1V_temp[n] = (dR_dw2V_temp[n].T@w2M)*sig_prime(Z1[n])    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw1V = np.sum(dR_dw1V_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    # Gradient der Matrix
    dR_dw1M_temp = np.zeros(shape = (d, 4, 2))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw1M_temp[n] = np.outer(dR_dw1V_temp[n], X0[n])    # Ermitteln des Gradienten für jeden Datenpunkt

    dR_dw1M = np.sum(dR_dw1M_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte
    
    
    ### Updaten der Gewichte ###
    
    # Schicht 1
    w1M = w1M-eta*dR_dw1M
    w1V = w1V-eta*dR_dw1V
    
    # Schicht 2
    w2M = w2M-eta*dR_dw2M
    w2V = w2V-eta*dR_dw2V
    
    # Schicht 3
    w3M = w3M-eta*dR_dw3M
    w3V = w3V-eta*dR_dw3V

## Auswerten eines Datenpunktes
Um das Ergebnis des trainierten neuronalen Netzes bestimmen zu können, muss eine Funktion mit der Hypothese mit der Theta-Funktion in letzter Instanz definiert werden.

__Aufgabe__: Vervollständige die nachstehende Funktion, um die Klassifikation durch das Netz durch ihren Aufruf bestimmen zu können.

In [ ]:
def h(x):
    return theta(z3(sig(z2(sig(z1(x))))))

## Auftragen der Ergebnisse

In der ersten nachfolgenden Zelle ist ganz links der vorliegende Datensatz zu sehen. Nach rechts fortschreitend sind dann die Aktivierungen der einzelnen Neuronen aufgetragen. Ganz rechts ist somit das Ergebnis des Neuronalen Netzes für die vorliegenden Datenpunkte zu sehen.

In der zweiten Zelle wird ein Contour-Plot mit den Ergebnissen des Neuronalen Netzes erstelln. Die vorliegenden Daten werden farblich markiert eingezeichnet.

In [ ]:
# Auftragen der Aktivierung in den einzelnen Schichten
plt.figure(figsize = (20, 20))

# Subplot für Eingabeschicht
plt.subplot(441)

plt.scatter(x1, x2, c = y, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')


# Subplot für Schicht 1, Neuron 1
plt.subplot(442)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[0]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 1, Neuron 2
plt.subplot(446)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[1]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 1, Neuron 3
plt.subplot(4, 4, 10)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[2]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 1, Neuron 4
plt.subplot(4, 4, 14)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[0]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 3, Neuron 1
plt.subplot(443)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z2(sig(z1(X0[n]))))[0]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 3, Neuron 2
plt.subplot(447)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z2(sig(z1(X0[n]))))[1]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Ausgabeschicht
plt.subplot(444)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = h(X0[n])

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')


plt.show()

In [ ]:
# Auftragung der Klassifikation des Netzes
plt.figure(figsize = (5, 5))

# Anlegen eines meshgrids
N = 100
X, Y = np.meshgrid(np.linspace(0, 1, N), np.linspace(0, 1, N))

h_eval = np.zeros(shape = (N, N))    # Leerer Array zum Speiechern er Klassifikation durch das Netz
    
for n in range(N):
    for m in range(N):
        h_eval[n][m] = h(np.array([X[n][m], Y[n][m]]))    # Ermitteln der Klassifikation durch das Netz
        
# Auftragen der Klassifikation durch das Netz
plt.contourf(X, Y, h_eval, cmap = 'seismic')

# Auftragen der Datenpunkte
plt.scatter(x1, x2, c = y , cmap = 'spring')

plt.show()

## Nicht linear separierbare Daten

Neuronale Netze mit mehreren Schichten (Deep Learning) sind in der Lage auch nicht linear separierbare Daten ohne händisches Feature Engineering klassifizieren zu können. Dazu sind im Datensatz `neural_network_data02.dat` nicht linear separierbare Daten zu finden.

__Aufgabe__: Trage den Datensatz auf und formatiere diesen, um ihn mit dem Neuronalen Netz klassifizieren zu können.

In [ ]:
# Laden und Auftragen der Daten
x1, x2, y = np.loadtxt("neural_network_data02.dat", unpack = True)

# Formatieren des Datensatzes
d = len(y)    # Bestimmen der Anzahl der Datenpunkte
X0 = np.zeros(shape = (d, 2))    # Erstellen eines leeren Arrays der passenden Form
for n in range(d):
    X0[n] = np.array([x1[n], x2[n]])    # Auffüllen des Arrays mit den entsprechenden Daten

# Auftragen des Datensatzes
plt.scatter(x1, x2, c = y, cmap = 'seismic')

plt.grid(True, color = 'grey', linestyle = '--')
plt.axis('equal')

plt.show()

## Trainieren eines Neuronal Netzes

Mit der gleichen Netzarchitektur wie oben beschrieben, kann auch diese Datensatz klassifiziert werden. Dazu bietet es sich jedoch an, die Gewichte mit zufälligen Werten aus dem Intervall $[-1, 1]$ zu initialiseren. 

__Aufgabe__: Trainiere ein Neuronales Netz, dass diesen Datensatz klassifziert. Prüfe mit den Auftragungen der Aktivierung und der Klassifikation, ob eine passende Klassifikation erfolgt.

In [ ]:
### Initialisieren der Gewichte ###

np.random.seed(15646)

# Schicht 1
w1V = np.random.rand(4)*2-1    # Verschiebungsvektor der Dimension 4
w1M = np.random.rand(4, 2)*2-1   # 4x2-Matrix

# Schicht 2
w2V = np.random.rand(2)*2-1   # Verschiebungsvektor der Dimension 2
w2M = np.random.rand(2, 4)*2-1    # 2x4-Matrix

# Schicht 3
w3V = np.random.rand()*2-1    # Verschiebungsvektor der Dimension 1 -> Zahl
w3M = np.random.rand(2)*2-1    # 1x2-Matrix, hier dargestellt als Vektor

### Einstellen der Hyperparameter ###
amplification = 5
epochen = 15000
eta = 10**(-3)

### Trainieren des Netzes ###
for i in range(epochen):    # Iterieren über die Epochen
    
    ### Forward Pass ###
    
    # Schicht 1
    Z1 = np.zeros(shape = (d, 4))    # Erstellen eines leeren Arrays zum Speichern von Z1 für jeden Datenpunkt
    for n in range(d):
        Z1[n] = z1(X0[n])    # Ermitteln von Z1 für jeden Datenpunkt
    X1 = sig(Z1)    # Ermitteln von X1 für jeden Datenpunkt
    
    
    # Schicht 2
    Z2 = np.zeros(shape = (d, 2))    # Erstellen eines leeren Arrays zum Speichern von Z2 für jeden Datenpunkt
    for n in range(d):
        Z2[n] = z2(X1[n])    # Ermitteln von Z2 für jeden Datenpunkt
    X2 = sig(Z2)    # Ermitteln von X2 für jeden Datenpunkt
    
    
    # Schicht 3
    Z3 = np.zeros(d)    # Erstellen eines leeren Arrays zum Speichern von Z3 für jeden Datenpunkt
    for n in range(d):
        Z3[n] = z3(X2[n])    # Ermitteln von Z3 für jeden Datenpunkt
    X3 = sig(Z3)    # Ermitteln von X3 für jeden Datenpunkt
    
    
    
    
    
    ### Backward Pass ###
    
    # Schicht 3
    # Gradient des Verschiebungsvektors
    dR_dw3V_temp = dR_dh(X3, y)*sig_prime(Z3)    # Bestimme einen Array mit den Gradienten der einzelnen Datenpunkten
    dR_dw3V = np.sum(dR_dw3V_temp)    # Summiere die einzelnen Gradienten auf
    
    # Gradient der Matrix
    dR_dw3M_temp = np.zeros(shape = (d, 2))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw3M_temp[n] = dR_dw3V_temp[n]*X2[n]    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw3M = np.sum(dR_dw3M_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    
    # Schicht 2
    # Gradient des Verschiebungsvektors
    dR_dw2V_temp = np.zeros(shape = (d, 2))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw2V_temp[n] = dR_dw3V_temp[n]*w3M*sig_prime(Z2[n])    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw2V = np.sum(dR_dw2V_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    # Gradient der Matrix
    dR_dw2M_temp = np.zeros(shape = (d, 2, 4))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw2M_temp[n] = np.outer(dR_dw2V_temp[n], X1[n])    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw2M = np.sum(dR_dw2M_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    
    # Schicht 1
    # Gradient des Verschiebungsvektors
    dR_dw1V_temp = np.zeros(shape = (d, 4))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw1V_temp[n] = (dR_dw2V_temp[n].T@w2M)*sig_prime(Z1[n])    # Ermitteln des Gradienten für jeden Datenpunkt
    dR_dw1V = np.sum(dR_dw1V_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte

    # Gradient der Matrix
    dR_dw1M_temp = np.zeros(shape = (d, 4, 2))    # Leerer Array zum Speichern des Gradienten für jeden Datenpunkt
    for n in range(d):
        dR_dw1M_temp[n] = np.outer(dR_dw1V_temp[n], X0[n])    # Ermitteln des Gradienten für jeden Datenpunkt

    dR_dw1M = np.sum(dR_dw1M_temp, axis = 0)    # Summieren dier einzelnen Gradienten, axis = 0 summiert nur über Datenpunkte
    
    
    ### Updaten der Gewichte ###
    
    # Schicht 1
    w1M = w1M-eta*dR_dw1M
    w1V = w1V-eta*dR_dw1V
    
    # Schicht 2
    w2M = w2M-eta*dR_dw2M
    w2V = w2V-eta*dR_dw2V
    
    # Schicht 3
    w3M = w3M-eta*dR_dw3M
    w3V = w3V-eta*dR_dw3V

In [ ]:
### Auftragen der Aktivierung in den einzelnen Schichten ###
plt.figure(figsize = (20, 20))

# Subplot für Eingabeschicht
plt.subplot(441)

plt.scatter(x1, x2, c = y, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')


# Subplot für Schicht 1, Neuron 1
plt.subplot(442)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[0]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 1, Neuron 2
plt.subplot(446)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[1]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 1, Neuron 3
plt.subplot(4, 4, 10)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[2]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 1, Neuron 4
plt.subplot(4, 4, 14)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z1(X0[n]))[0]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 3, Neuron 1
plt.subplot(443)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z2(sig(z1(X0[n]))))[0]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Schicht 3, Neuron 2
plt.subplot(447)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = sig(z2(sig(z1(X0[n]))))[1]

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')

# Subplot für Ausgabeschicht
plt.subplot(444)

h_eval = np.zeros(d)
for n in range(d):
    h_eval[n] = h(X0[n])

plt.scatter(x1, x2, c = h_eval, cmap = 'seismic')
plt.axis('equal')
plt.grid(True, color = 'grey', linestyle = '--')


plt.show()

In [ ]:
### Auftragung der Klassifikation des Netzes ###
plt.figure(figsize = (5, 5))

# Anlegen eines meshgrids
N = 100
X, Y = np.meshgrid(np.linspace(-1, 1, N), np.linspace(-1, 1, N))

h_eval = np.zeros(shape = (N, N))    # Leerer Array zum Speiechern er Klassifikation durch das Netz
    
for n in range(N):
    for m in range(N):
        h_eval[n][m] = h(np.array([X[n][m], Y[n][m]]))    # Ermitteln der Klassifikation durch das Netz
        
# Auftragen der Klassifikation durch das Netz
plt.contourf(X, Y, h_eval, cmap = 'seismic')

# Auftragen der Datenpunkte
plt.scatter(x1, x2, c = y , cmap = 'spring')

plt.show()